In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import tqdm

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from scipy import exp
from scipy.optimize import curve_fit

import pandas as pd
import os
import shutil
import cv2

import glob2

from numba import jit
import tables
from scipy.io import loadmat
import scipy
import h5py
import hdf5storage
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.cm as cm

from utility_simulated_drift import (make_shifted_templates,
                                      make_default_template,
                                      draw_template,
                                      gaus,
                                      make_drift_template,
                                      shift_templates_all,
                                      WaveForms,
                                      generate_poisson_uniform_firingrate2,
                                      select_ground_truth_units,
                                      visualize_drift,
                                      generate_synthetic_data,
                                      visualize_traces,
                                      make_full_data)

colors = [
'black','grey','brown','slategrey',    
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']
clrs= colors


Autosaving every 180 seconds


In [2]:
# select working directory location
root_dir = '/media/cat/4TBSSD/data/synthetic/run21_random_walk_5Hz_10Hz/'

try:
    os.mkdir(root_dir)
except:
    pass

# set radius of channels to consider from maximum channel outwards (in micrometers)
radius = 200

# select ground truth templates to be used
#fname_templates = '/media/cat/12TB/dbox/Dropbox/code/neuropixels/allen_inst_2ndrec/templates_reloaded.npy'
#geom = np.loadtxt('/home/cat/p1_g0_t0.imec0.ap_geom_384chans.txt')
fname_templates = '/media/cat/4TBSSD/data/synthetic/allen_inst_2ndrec/templates_reloaded.npy'
geom = np.loadtxt('/media/cat/4TBSSD/data/synthetic/p1_g0_t0.imec0.ap_geom.txt')


# select largest X templates to be used for injection
n_templates = 100
(units, templates_ground_truth) = select_ground_truth_units(root_dir, 
                                                            fname_templates,
                                                            n_templates,
                                                            geom)

print ("templates ground truth: ", templates_ground_truth.shape)

Total templates avialable;  (575, 101, 384)
Total units selected:  100 , ids:  [420 339 421 348 473 418  62 362 565 239] ...
templates ground truth:  (100, 101, 384)


In [3]:
# ids = np.load('/home/cat/Downloads/spike_clusters(1).npy')
# print (ids.shape)
# data= np.load('/home/cat/Downloads/amplitudes.npy')
# print (data.shape)

In [4]:

# for k in range(25):
#     ax=plt.subplot(5,5,k+1)
#     idx = np.where(ids==k)[0]
#     plt.scatter(np.arange(idx.shape[0]), data[idx],alpha=.02)
#     plt.title("Unit: "+str(k),fontsize=10)
# plt.suptitle("Allen Inst data 8084036578, first 25 neurons",fontsize=20)
# plt.show()

In [3]:
# visualize the desired shifts on a particular unit
#shifts = np.arange(0., 2.55, 0.05)
#shifts = np.array([-2, -1.5, -1.0, -0.5, 0])
#shifts = np.arange(0, .5, 0.025)

# random walk 
shifts = np.random.normal(0,1,size=60)*.2
print ("shifts: ", shifts)
plt.plot(np.cumsum(shifts))
shifts = np.cumsum(shifts)

#shifts = np.hstack((shifts, np.array([0,1.6, 2.3])))
#shifts = np.array([-2.0, -1.99, 1.99, 2.0])
print (shifts)

# visualize drifted template for any particular unit 
unit_id = 50
visualize_drift(shifts, 
                geom, 
                templates_ground_truth, 
                unit_id,
                radius)


shifts:  [ 0.0130806  -0.2035754  -0.23384795  0.03465429  0.12732944 -0.05226094
  0.21951865  0.08264281  0.44384605 -0.28518941 -0.36875628 -0.02275811
  0.06104145 -0.05684748 -0.05431091  0.10941102 -0.12892384 -0.0221501
  0.07135496 -0.09316327 -0.31252006  0.27718355 -0.12135653  0.31533515
  0.09714615  0.21511586  0.0615218   0.274085   -0.14989654  0.3576128
  0.10718877 -0.14139572  0.07272135  0.20602285  0.08508012  0.26761525
 -0.35295338  0.37356589 -0.12210832 -0.13280099  0.05134519  0.26871738
 -0.22752639 -0.00885258 -0.24588361  0.21622203  0.35217513  0.03834471
  0.30035563 -0.02202775 -0.09362349  0.18742271  0.2472778  -0.08124232
 -0.13680096 -0.21433502 -0.03869596 -0.11525876 -0.23945456  0.12999975]
[ 0.0130806  -0.1904948  -0.42434276 -0.38968847 -0.26235902 -0.31461996
 -0.09510131 -0.0124585   0.43138755  0.14619814 -0.22255815 -0.24531625
 -0.18427481 -0.24112228 -0.29543319 -0.18602218 -0.31494602 -0.33709612
 -0.26574116 -0.35890444 -0.6714245  -0.394

/home/cat/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/home/cat/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [4]:
# generate synthetic drift data
sample_rate = 30000
rec_len_sec = 60  # seconds
rec_length_sample_times = rec_len_sec * sample_rate
firingrate_min = 5
firingrate_max = 10


# generate synthetic data
data_synthetic, windows = generate_synthetic_data(root_dir,
                                         rec_len_sec, 
                                         sample_rate, 
                                         shifts, 
                                         units, 
                                         templates_ground_truth,
                                         radius,
                                         geom,
                                         firingrate_min,
                                         firingrate_max)

np.savetxt(root_dir+'/ground_truth/shifts.txt', shifts)
print (data_synthetic.shape)


(1800000, 384)
[      0.           30508.47457627   61016.94915254   91525.42372881
  122033.89830508  152542.37288136  183050.84745763  213559.3220339
  244067.79661017  274576.27118644  305084.74576271  335593.22033898
  366101.69491525  396610.16949153  427118.6440678   457627.11864407
  488135.59322034  518644.06779661  549152.54237288  579661.01694915
  610169.49152542  640677.96610169  671186.44067797  701694.91525424
  732203.38983051  762711.86440678  793220.33898305  823728.81355932
  854237.28813559  884745.76271186  915254.23728814  945762.71186441
  976271.18644068 1006779.66101695 1037288.13559322 1067796.61016949
 1098305.08474576 1128813.55932203 1159322.03389831 1189830.50847458
 1220338.98305085 1250847.45762712 1281355.93220339 1311864.40677966
 1342372.88135593 1372881.3559322  1403389.83050847 1433898.30508475
 1464406.77966102 1494915.25423729 1525423.72881356 1555932.20338983
 1586440.6779661  1616949.15254237 1647457.62711864 1677966.10169492
 1708474.57627119 17

 inserting unit:  0 [493291 507973 508475 515469] [1.00811362 1.03869154 0.84280348 1.02954573]
 inserting unit:  50 [488409 503088 503180 506032 514819 514858] [0.90614421 0.97180255 0.95576174 1.03313414 0.98834573 0.94053209]
************


window:  [518644.0677966102, 549152.5423728813] , shift:  -0.33709612047425036  (inter channel units)
 inserting unit:  0 [520735 520847 524789 527266 528894 529070 529495 536516 537404 548926] [0.95164454 0.90948367 1.02358051 0.95958305 0.86027213 0.99781396
 0.97573539 0.96493693 0.94289011 0.95969807]
 inserting unit:  50 [518916 521814 522457 535512 540633] [1.00171758 0.96030466 0.86087286 0.97915145 1.05108385]
************


window:  [549152.5423728813, 579661.0169491526] , shift:  -0.26574116182906166  (inter channel units)
 inserting unit:  0 [552060 556467 559272 559984 564188 565628 566724 566873 567963 569325] [1.02368793 1.04423617 0.92799625 1.01067911 1.05790955 1.01703288
 0.86992843 1.00019777 0.95601642 0.9655552 ]
 inserting u

 inserting unit:  0 [1099861 1100181 1102577 1106157 1115328 1119872] [1.01339048 0.9211171  1.01549557 1.00518398 0.91145626 0.91311361]
 inserting unit:  50 [1098537 1105552 1106216 1106871 1108825 1114032 1117703 1121281 1126520
 1127968] [1.08447081 1.04878678 1.01427566 0.83756896 0.99831044 0.98102745
 0.91620904 0.94722377 0.96590148 1.02480049]
************


window:  [1128813.559322034, 1159322.0338983051] , shift:  1.273167871162789  (inter channel units)
 inserting unit:  0 [1137307 1139306 1139483 1144761 1146880 1149351 1149993 1155436 1156271] [1.04618793 0.89307217 0.84940611 0.98738825 0.90374524 1.01278354
 0.99970742 0.96836467 0.8842427 ]
 inserting unit:  50 [1147118 1148158 1152044 1152427 1156169] [0.98373054 0.9785826  0.91140392 0.88099808 0.91707399]
************


window:  [1159322.0338983051, 1189830.5084745763] , shift:  1.1510595470869143  (inter channel units)
 inserting unit:  0 [1162382 1166365 1169568 1170395 1176634 1177121 1182997 1184483 1186564
 118

 inserting unit:  0 [1649445 1650170 1660027 1666270 1668434] [1.01300075 0.92463523 1.07152356 0.99724503 0.89113705]
 inserting unit:  50 [1648058 1650822 1659565 1664859 1665005 1668138 1669896 1670311 1672492
 1672700] [0.9018565  1.04207903 1.00881628 1.08028171 0.99698048 0.97164089
 1.00616489 0.82758842 0.96614478 0.95942215]
************


window:  [1677966.1016949152, 1708474.5762711863] , shift:  1.6498270098215482  (inter channel units)
 inserting unit:  0 [1678742 1681895 1687542 1691656 1694111 1700948 1702611] [1.02822897 1.01283272 1.08359953 1.07730698 0.93672114 0.92726249
 0.89313702]
 inserting unit:  50 [1681449 1683763 1689130 1690167 1698680 1703055 1706232] [1.06567957 0.99705021 1.03682688 1.01534816 1.0340398  1.04811507
 0.94434016]
************


window:  [1708474.5762711863, 1738983.0508474577] , shift:  1.61113104970672  (inter channel units)
 inserting unit:  0 [1710228 1711227 1713079 1716676 1727914 1732927 1734999] [1.0404699  0.94045469 1.01659841 0.9

In [45]:
# visualize the final traces
visualize_traces(data_synthetic, 0, 5000)


In [5]:
print (len(windows), len(shifts))
drift = np.zeros((data_synthetic.shape[0]),'float32')
print (drift.shape)
for k in range(len(windows)):
    print (k, windows[k])
    drift[int(windows[k][0]):int(windows[k][1])]=shifts[k]

plt.plot(drift)
plt.show()

59 60
(1800000,)
0 [0.0, 30508.474576271186]
1 [30508.474576271186, 61016.94915254237]
2 [61016.94915254237, 91525.42372881356]
3 [91525.42372881356, 122033.89830508475]
4 [122033.89830508475, 152542.37288135593]
5 [152542.37288135593, 183050.84745762713]
6 [183050.84745762713, 213559.3220338983]
7 [213559.3220338983, 244067.7966101695]
8 [244067.7966101695, 274576.27118644066]
9 [274576.27118644066, 305084.74576271186]
10 [305084.74576271186, 335593.22033898305]
11 [335593.22033898305, 366101.69491525425]
12 [366101.69491525425, 396610.16949152545]
13 [396610.16949152545, 427118.6440677966]
14 [427118.6440677966, 457627.1186440678]
15 [457627.1186440678, 488135.593220339]
16 [488135.593220339, 518644.0677966102]
17 [518644.0677966102, 549152.5423728813]
18 [549152.5423728813, 579661.0169491526]
19 [579661.0169491526, 610169.4915254237]
20 [610169.4915254237, 640677.966101695]
21 [640677.966101695, 671186.4406779661]
22 [671186.4406779661, 701694.9152542372]
23 [701694.9152542372, 7322

In [6]:
# Make a correlated data chunk of 10 seconds 
# longer takes too long of time; shorter will add more boundary artifcats to recording
time = 10*sample_rate
print ("length of correlated noise snipit: ", time)

fname = root_dir+'correlated_noise.npy'
fname = '/media/cat/4TBSSD/data/synthetic/run18/correlated_noise.npy'
# use ground_truth_templates to make noise
if os.path.exists(fname)==False:
    temps = templates_ground_truth.copy()
    WF = WaveForms(temps.transpose(0,2,1))
    correlated_noise = WF.generate_correlated_noise_add(time)

    print ("DONE MAKING CORRELATED NOISE for 10 sec chunk")
    
    np.save(fname, correlated_noise)
else:
    correlated_noise = np.load(fname)

length of correlated noise snipit:  300000


In [7]:
# blend correlated noise and white noise with injected spike dataset
data_sum = make_full_data(data_synthetic, rec_length_sample_times, 
                          sample_rate, root_dir,
                          correlated_noise)


white noise:  (300000, 384)
time chunk:  0  to  300000
time chunk:  300000  to  600000
time chunk:  600000  to  900000
time chunk:  900000  to  1200000
time chunk:  1200000  to  1500000
time chunk:  1500000  to  1800000


In [8]:
# visualize first 10000 time points;
visualize_traces(data_sum, 0, 10000)


In [10]:
print (data_sum.shape)

(1800000, 384)


In [9]:
import numpy as np
import h5py

#mystuff = np.random.rand(10,30)

f = h5py.File(root_dir+'/data.h5', 'w')
f['rawdata'] = data_sum
f['geometry'] = geom
f['drift'] = drift
f.close()

In [9]:
print (len(shifts))
print (len(windows))

40
39


In [7]:
# visualize spike triggered max chan data
spike_train_gt = np.load('/media/cat/1TB/data/synthetic/run11/ground_truth/spike_train_ground_truth.npy')
fname= '/media/cat/1TB/data/synthetic/run11/data_int16.bin'

from utility_match_units import binary_reader_waveforms

In [14]:
n_channels = 384
n_times = 101

# 
units = np.arange(25)
units = [13]
for unit in units:
    #ax=plt.subplot(5,5,unit+1)
    idx = np.where(spike_train_gt[:,1]==unit)[0]
    spikes = spike_train_gt[idx,0]
    wfs = binary_reader_waveforms(fname, n_channels, n_times, spikes, data_type='int16')

    print (wfs.shape)
    max_chan = wfs.mean(0).ptp(0).argmax(0)

    #fig=plt.figure()
    plt.plot(wfs[:,:,max_chan].T,c='black')
    plt.show()

    if True:
        fig=plt.figure()
        for k in range(wfs.shape[0]):
            ax=plt.subplot(10,10,k+1)
            plt.plot(wfs[k,:,max_chan].T,c='black')
            plt.title(str(k)+" "+str(spikes[k]))
        plt.show()

(85, 101, 384)


In [15]:
times = np.load('/media/cat/1TB/temp/'+str(unit)+'_times.npy')
scale = np.load('/media/cat/1TB/temp/'+str(unit)+'_scale.npy')

print (times.shape)
print (scale.shape)
print (scale)


(85,)
(85,)
[1.         0.69767633 0.64403642 1.         1.         0.99501248
 0.74453159 0.71177032 0.68045064 1.         1.         0.89136614
 0.79057085 1.         1.         1.         0.68728928 1.
 0.81873075 1.         1.         0.93706746 0.86070798 0.83945702
 1.         0.97044553 0.72614904 0.86935824 0.85641518 0.98019867
 1.         0.66034028 1.         0.62188506 1.         0.99004983
 0.69073433 0.77105159 0.83110428 0.70117344 1.         0.99004983
 1.         0.88692044 0.81058425 1.         0.61878339 0.92774349
 1.         0.63762815 1.         0.6156972  1.         1.
 0.68728928 0.78662786 1.         0.60957091 0.60957091 0.77880078
 0.85214379 1.         1.         1.         1.         0.72614904
 0.70117344 1.         1.         1.         0.66697681 0.99501248
 1.         1.         0.62188506 0.81873075 0.62500227 0.66034028
 0.65376979 1.         0.82695913 0.64082428 0.64082428 1.
 1.        ]
